In [11]:
import lyricsgenius
import pandas as pd
import time
import requests

# API setup 

DISCOGS_TOKEN = "xmWNqVMtvKoFllImiTkEbgIMpgmdVRVggXqrWJPQ"
GENIUS_API_TOKEN = "r6vW9hyUck05RRuvBF4L1f5MjirVAeuHO0tauARd-aB1pt7xscCUVzMnFoFC2WTS"

# Genius setup 
genius = lyricsgenius.Genius(GENIUS_API_TOKEN)
genius.verbose = False
genius.remove_section_headers = True
genius.skip_non_songs = True
genius.retries = 3

# Discogs song data

def search_discogs_by_year(year, per_page=10, page=1):
    url = "https://api.discogs.com/database/search"
    params = {
        "year": year,
        "type": "release",
        "per_page": per_page,
        "page": page,
        "token": DISCOGS_TOKEN
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Discogs API error: {response.status_code}")
        return []
    return response.json().get("results", [])

# Lyrics data

def get_lyrics(title, artist):
    try:
        song = genius.search_song(title=title, artist=artist)
        if song:
            return song.lyrics
    except Exception as e:
        print(f"Lyrics fetch failed: {title} by {artist} - {e}")
    return None

# Put into df fomrat
def build_dataset_from_discogs(year, max_releases=10):
    print(f"\nSearching Discogs releases for {year}")
    results = search_discogs_by_year(year, per_page=max_releases)
    songs = []

    for release in results:
        try:
            title = release.get("title", "")
            split_title = title.split(" - ", 1)
            if len(split_title) < 2:
                continue
            artist, track_title = split_title
            artist = artist.strip()
            track_title = track_title.strip()

            song_data = {
                "title": track_title,
                "artist": artist,
                "year": year,
                "country": release.get("country"),
                "genres": ";".join(release.get("genre", [])),
                "styles": ";".join(release.get("style", []))
            }

            print(f"Fetching lyrics: {track_title} by {artist}")
            lyrics = get_lyrics(track_title, artist)
            if lyrics:
                song_data["lyrics"] = lyrics
                songs.append(song_data)

            time.sleep(1)
        except Exception as e:
            print(f"Error processing release: {e}")
            continue

    return pd.DataFrame(songs)

In [13]:
# Testing

df = build_dataset_from_discogs(2008, max_releases=5)
print("\nSample data:")
print(df.head())


📅 Searching Discogs releases for 2008
🎵 Fetching lyrics: Junglist (98 Remix) / Junglist (Original Dubplate) by Tribe Of Issachar
🎵 Fetching lyrics: Cult Of Destruction by Perversor
🎵 Fetching lyrics: Digital Shades [Vol. I] by M83
🎵 Fetching lyrics: Settling Son (Demo) by Less Than Jake
🎵 Fetching lyrics: 4th Bouquet by Various

✅ Sample data:


KeyError: "None of [Index(['title', 'artist', 'year', 'country', 'genres', 'styles'], dtype='object')] are in the [columns]"

In [15]:
df.head()

""


In [ ]:
# Save to CSV ...eventually
#df.to_csv("discogs_genius_2008.csv", index=False)